In [ ]:
import numpy as np  
import psutil
import gc   # garbage collector

# Free up memory if previous data exists
gc.collect()

# Get available memory in MB before loading new data
available_memory_before = psutil.virtual_memory().available / (1024 * 1024)
print(f"Remaining Memory Before Loading Data: {available_memory_before:.2f} MB")

# Load preprocessed data
X_train_scaled = np.load('../_data/X_train_scaled.npy')
X_test_scaled = np.load('../_data/X_test_scaled.npy')
Y_train = np.load('../_data/Y_train.npy')
Y_test = np.load('../_data/Y_test.npy')

# Get available memory after loading data
available_memory_after = psutil.virtual_memory().available / (1024 * 1024)
print(f"Remaining Memory After Loading Data: {available_memory_after:.2f} MB")

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, r2_score
import seaborn as sns
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold



# Convert labels to categorical if needed
num_classes = len(np.unique(Y_train))
Y_train_cat = to_categorical(Y_train, num_classes)
Y_test_cat = to_categorical(Y_test, num_classes)

# Define model function
def build_model(input_shape, num_classes):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze EfficientNet layers
    
    x = base_model.output
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Training with K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

epochs = 20
history_all = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_scaled)):
    print(f"Training Fold {fold+1}")
    X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
    Y_train_fold, Y_val_fold = Y_train_cat[train_idx], Y_train_cat[val_idx]
    
    model = build_model(X_train_scaled.shape[1:], num_classes)
    history = model.fit(X_train_fold, Y_train_fold, epochs=epochs, validation_data=(X_val_fold, Y_val_fold), batch_size=32, verbose=1)
    
    val_acc = max(history.history['val_accuracy'])
    fold_accuracies.append(val_acc)
    history_all.append(history)

# Final Evaluation
model = build_model(X_train_scaled.shape[1:], num_classes)
model.fit(X_train_scaled, Y_train_cat, epochs=epochs, batch_size=32, verbose=1)
test_loss, test_acc = model.evaluate(X_test_scaled, Y_test_cat, verbose=0)
print(f"Test Accuracy: {test_acc*100:.2f}%")

# Predictions
Y_pred = model.predict(X_test_scaled)
Y_pred_classes = np.argmax(Y_pred, axis=1)
Y_true = np.argmax(Y_test_cat, axis=1)

# Confusion Matrix
plt.figure(figsize=(8,6))
cm = confusion_matrix(Y_true, Y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Classification Report
print("Classification Report:\n", classification_report(Y_true, Y_pred_classes))

# R² Score
r2 = r2_score(Y_true, Y_pred_classes)
print(f"R² Score: {r2:.2f}")
plt.bar(['R2 Score'], [r2], color='blue')
plt.ylim(0, 1)
plt.title('R² Score')
plt.show()

# F1 Score Plot
f1_scores = classification_report(Y_true, Y_pred_classes, output_dict=True)['weighted avg']['f1-score']
plt.bar(['F1 Score'], [f1_scores], color='green')
plt.ylim(0, 1)
plt.title('F1 Score')
plt.show()

# Train/Test Accuracy & Loss Curves
plt.figure(figsize=(12,5))
for i, history in enumerate(history_all):
    plt.plot(history.history['accuracy'], label=f'Train Fold {i+1}')
    plt.plot(history.history['val_accuracy'], label=f'Val Fold {i+1}')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Train & Validation Accuracy Across Folds')
plt.grid()
plt.show()

# ROC Curve
plt.figure(figsize=(8,6))
for i in range(num_classes):
    fpr, tpr, _ = roc_curve(Y_test_cat[:, i], Y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i} (AUC = {roc_auc:.2f})')
plt.plot([0,1], [0,1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()

print("Training and evaluation complete.")


In [ ]:
model.save('efficientnetb0.h5')

#End